# Using a pre-trained VGG-16 model
![](http://karpathy.github.io/assets/cnntsne.jpeg)

In [1]:
# Usual
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

# Autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
% matplotlib inline

# Seaborn config
import seaborn as sns
sns.set(style='whitegrid', color_codes=True)
sns.set(font_scale=1)

In [2]:
# Images
from PIL import Image
from io import BytesIO
import scipy
import requests

# Keras
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

Using TensorFlow backend.


# VGG-16
![](https://www.cs.toronto.edu/~frossard/post/vgg16/vgg16.png)

[Link to the original paper](https://arxiv.org/abs/1409.1556)

# Fetching the pre-tained model with Keras

In [3]:
model = VGG16(weights='imagenet', include_top=True)

##### Looking at the different layers

In [4]:
layers = dict([(layer.name, layer.output) for layer in model.layers])
layers

{'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(?, 224, 224, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(?, 224, 224, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(?, 112, 112, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(?, 112, 112, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(?, 112, 112, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(?, 56, 56, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(?, 56, 56, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(?, 56, 56, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(?, 56, 56, 256) dtype=float32>,
 'block3_pool': <tf.Tensor 'block3_pool/MaxPool:0' shape=(?, 28, 28, 256) dtype=float32>,
 'block4_conv1': <tf.Tensor 'block4_conv1/Relu:0' shape=(?, 28, 28, 512) dtype=float32>,
 'block4_co

##### How many parameters do we have ?

In [5]:
model.count_params()

138357544

# Using the VGG-16 model on sample images

## Designing a function to test a web image

On this picture : 
![](http://www.gettyimages.fr/gi-resources/images/Embed/new/embed2.jpg)

##### Getting the image data from the url

In [6]:
example_url = "http://www.gettyimages.fr/gi-resources/images/Embed/new/embed2.jpg"

In [7]:
def read_image_from_url(url,colors = True,resize = None):
    """
    IO function that reads a picture from an url
    Takes as input : 
        - The url
        - colors = True returns a 3 dimensional array, False a two dimensional array for the picture in black & white
        - resize a tuple to resize the picture at a given pixels shape
    Returns: a numpy array for the pixel value of the image
    """
    request = requests.get(url)
    if request.ok:
        img = scipy.misc.imread(BytesIO(request.content),flatten = not colors)
        if img.shape[-1] == 4:
            img = img[:,:,range(3)]            
        if resize is not None:
            img = scipy.misc.imresize(img,resize,'cubic')
        return img
    else:
        print("Invalid request")

        
img = read_image_from_url(example_url)
print(img.shape)
img[:5,:5,:]

(359, 500, 3)


array([[[167, 142, 101],
        [167, 142, 101],
        [167, 142, 101],
        [168, 143, 102],
        [168, 143, 102]],

       [[167, 142, 101],
        [167, 142, 101],
        [168, 143, 102],
        [168, 143, 102],
        [168, 143, 102]],

       [[168, 143, 102],
        [168, 143, 102],
        [168, 143, 102],
        [169, 144, 103],
        [169, 144, 103]],

       [[169, 144, 103],
        [169, 144, 103],
        [169, 144, 103],
        [169, 144, 103],
        [170, 145, 104]],

       [[169, 144, 103],
        [170, 145, 104],
        [170, 145, 104],
        [170, 145, 104],
        [171, 146, 105]]], dtype=uint8)

##### Preprocess the image to work with the model

In [8]:
def preprocess_image(img):
    img = scipy.misc.imresize(img,(224,224),'cubic')
    img = np.asarray(img,dtype = "float32")
    img = np.expand_dims(img,axis = 0)
    img = preprocess_input(img)
    return img

x = preprocess_image(img)

##### Prediction

In [9]:
prediction = model.predict(x)

Probabilities are given for each of the 1000 class : 

In [10]:
prediction.shape

(1, 1000)

The prediction was quite correct !

In [11]:
decode_predictions(prediction, top=3)

[[('n02124075', 'Egyptian_cat', 0.44606611),
  ('n02123045', 'tabby', 0.075424992),
  ('n02123159', 'tiger_cat', 0.071717076)]]

##### Big function !

In [62]:
def predict_image(url):
    img = read_image_from_url(url)
    x = preprocess_image(img)
    prediction = model.predict(x)
    return decode_predictions(prediction,top = 3)

## Let's try that on other images

### Some successes

##### Monkey
![](http://pngimg.com/uploads/monkey/monkey_PNG18736.png)

In [76]:
predict_image("http://pngimg.com/uploads/monkey/monkey_PNG18736.png")

[[('n02481823', 'chimpanzee', 0.91695374),
  ('n02487347', 'macaque', 0.038173083),
  ('n02493793', 'spider_monkey', 0.024044253)]]

##### Plane
![](https://ichef-1.bbci.co.uk/news/660/cpsprodpb/0CB0/production/_89284230_planepa.jpg)

In [80]:
predict_image("https://ichef-1.bbci.co.uk/news/660/cpsprodpb/0CB0/production/_89284230_planepa.jpg")

[[('n02690373', 'airliner', 0.83158499),
  ('n04592741', 'wing', 0.16258174),
  ('n04552348', 'warplane', 0.0041271355)]]

### Some failures

##### Face

![](https://images-na.ssl-images-amazon.com/images/I/41ZN1VIePUL.jpg)

In [78]:
predict_image("https://images-na.ssl-images-amazon.com/images/I/41ZN1VIePUL.jpg")

[[('n03814639', 'neck_brace', 0.12140641),
  ('n02883205', 'bow_tie', 0.10820005),
  ('n02669723', 'academic_gown', 0.085149564)]]

##### Woman
![](http://www.wethepeoplestyle.com/wp-content/uploads/2016/02/jessiebush_wethepeople_streetstyle_nyfw7-815x545.jpg)

In [77]:
predict_image("http://www.wethepeoplestyle.com/wp-content/uploads/2016/02/jessiebush_wethepeople_streetstyle_nyfw7-815x545.jpg")

[[('n04479046', 'trench_coat', 0.14009823),
  ('n03891332', 'parking_meter', 0.10048288),
  ('n03141823', 'crutch', 0.07991156)]]